**Notebook 9: Recursos adicionales**

Objetivo: Recopilación de trozos de código de ánalisis NLP que fueron evaluados pero no utilizados en la versión final 
    del estudio. No obstante, pueder ser de gran utilidad para otras tareas específicas. 
    

In [3]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from spacy.matcher import PhraseMatcher
from scipy import spatial
import spacy
import nltk
from gensim.models import LdaModel
from gensim.models import CoherenceModel
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import Dictionary
import nltk
import spacy
from fuzzy_sentences_clustering import look_for_clusters
import requests
from bs4 import BeautifulSoup
import spacy
import torch
from transformers import BertTokenizerFast, EncoderDecoderModel
from nltk.corpus import wordnet as wn

In [4]:
# Directorios [cambiar a directorio propios]
pme_path = r'C:\Users\JoaquinFarina\Dropbox\Team NLP\Data\Planes de Mejoramiento Educativo\InformeFinal'
# establecer directorio de trabajo
os.chdir(pme_path)

In [5]:
# cargar bbdd
df_planificacion_implementacion_total = pd.read_csv(r'Datos\Finales\Nivel_accion\df_planificacion_implementacion_total_2018_2022_ACCIONES_FINAL.csv', low_memory=False)

In [17]:
df_planificacion_implementacion_total.columns

Index(['agno', 'rbd', 'region', 'deprov', 'comuna', 'dependencia',
       'nombre_actividad', 'en_planificacion_anual', 'dimensión_planif',
       'subdimensión_planif', 'objetivo_planif', 'estrategia_planif',
       'descripción_del_plan_planif', 'fecha_inicio_planif',
       'fecha_término_planif', 'programa_asociado_planif',
       'nombre_responsable_planif', 'ate_planif', 'planes_planif',
       'en_reporte_implementacion', 'dimensión_implem', 'subdimensión_implem',
       'objetivo_implem', 'estrategia_implem', 'descripción_del_plan_implem',
       'fecha_inicio_implem', 'fecha_término_implem',
       'programa_asociado_implem', 'nombre_responsable_implem', 'ate_implem',
       'planes_implem', 'monto_subvencion_general', 'monto_sep', 'monto_pie',
       'monto_mantenimiento', 'monto_pro_retencion', 'monto_internado',
       'monto_reforzamiento', 'monto_feap', 'monto_aporte_municipal',
       'monto_total', 'nombre_nivel_ejecucion',
       'nombre_justificacion_nivel_ejecucion',

In [18]:
for col in df_planificacion_implementacion_total.columns:
    print(col)

agno
rbd
region
deprov
comuna
dependencia
nombre_actividad
en_planificacion_anual
dimensión_planif
subdimensión_planif
objetivo_planif
estrategia_planif
descripción_del_plan_planif
fecha_inicio_planif
fecha_término_planif
programa_asociado_planif
nombre_responsable_planif
ate_planif
planes_planif
en_reporte_implementacion
dimensión_implem
subdimensión_implem
objetivo_implem
estrategia_implem
descripción_del_plan_implem
fecha_inicio_implem
fecha_término_implem
programa_asociado_implem
nombre_responsable_implem
ate_implem
planes_implem
monto_subvencion_general
monto_sep
monto_pie
monto_mantenimiento
monto_pro_retencion
monto_internado
monto_reforzamiento
monto_feap
monto_aporte_municipal
monto_total
nombre_nivel_ejecucion
nombre_justificacion_nivel_ejecucion
rural_rbd
básica_adultos
básica_niños
media_hc_adultos
media_hc_jóvenes
media_tp_y_artística_adultos
media_tp_y_artística_jóvenes
parvularia
parvularia_ind
básica_niños_ind
media_hc_jóvenes_ind
media_tp_y_artística_jóvenes_ind
básica

# Text summarization

https://huggingface.co/mrm8488/bert2bert_shared-spanish-finetuned-summarization

In [ ]:
# Para analisis a nivel de PME (en lugar de acción)
# Unir nombre de actividad y descripción 
df_planificacion_implementacion_total['nombre_y_descripción_actividad'] = df_planificacion_implementacion_total[['nombre_actividad', 'descripción_del_plan']].agg(': '.join, axis=1)

df_planificacion_implementacion_total['nombre_y_descripción_actividad'] = df_planificacion_implementacion_total['nombre_y_descripción_actividad'].astype(str) + '\n'

# Agrupar por Objeetivo y estrategia 
grp = df_planificacion_implementacion_total.groupby(['rbd','objetivo', 'estrategia']).agg(C=('nombre_y_descripción_actividad', ', '.join)).reset_index()


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = 'mrm8488/bert2bert_shared-spanish-finetuned-summarization'
tokenizer = BertTokenizerFast.from_pretrained(ckpt)
model = EncoderDecoderModel.from_pretrained(ckpt).to(device)

def generate_summary(text):
 
    inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    output = model.generate(input_ids, attention_mask=attention_mask)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
# Ejemplo
text = 'día de la convivencia con la familia: se realizara una actividad de convivencia el día jueves 30 de mayo, en reunión de padres y apoderados, donde los estudiantes compartirán con sus familias, profesores y compañeros disfrutando de un espectáculo musical junto a la comunidad educativa, a cada nivel se les entregara 2 tortas grandes mas 2 bebidas de 3 lts. cada una para favorecer convivencia de cada nivel.\n'
generate_summary(text)

# Sinónimos

Permite calcular sinónimos de palabras clave PME, mediante dos procedimientos distintos

In [ ]:
# cargar categorizaciones pre establecidas 
keywords = pd.read_excel(path + r'\Planes de Mejoramiento Educativo\Listado palabras y conceptos Estudio PME FORMATEADO (Febrero).xlsx')

In [ ]:
keyword_dict = {}
for ind, row in keywords.iterrows():
    text = str(row['Palabras/Conceptos'])
    text = text.replace('”', '"')
    text = text.replace('“', '"')
    print(ind)
    keyword_dict[ind] = text.split('"')[1::2]

## método 1: NLTK corpus

In [ ]:
def calcula_sinonimos(palabra, idioma='spa'):
    ''' Función que calcula las palabras sinónimas de una palabra dada
    '''
    sinonimos = []                                   # Conjunto de sinonimos inicialmente vacío
    for syn in wn.synsets(palabra, lang=idioma):     # Para cada synset en el que aparezca la palabra
        for lema in syn.lemmas(lang=idioma):         # Para cada lema de cada synset
            sinonimos+=[lema.name()]               # Se añade el lema (en el atributo 'name') al conjunto
    return sinonimos                                 # Se devuelve el conjunto de sinónimos

In [ ]:
# download language model: python -m spacy download es_core_news_sm 
nlp = spacy.load("es_core_news_sm")

In [ ]:
list_sinonimos = []
keyterms = []
for j in range(51):
    for word in keyword_dict[j]:
        print(word)
        if len(word.split())==1:
            doc = nlp(word)
            lemmas = [token.lemma_ for token in doc]
            for lemma in lemmas:
                print(lemma)
                keyterms += [lemma]
                list_sinonimos += [[x for x in calcula_sinonimos(lemma) if lemma !=x and lemma.lower() !=x]]
                print([x for x in calcula_sinonimos(lemma) if lemma !=x and lemma.lower() !=x])

In [ ]:
df_sinonimos = pd.DataFrame(list(zip(keyterms,list_sinonimos)),
               columns =['Palabra clave', 'Sinónimos'])
df_sinonimos['N_sinonimos'] = df_sinonimos['Sinónimos'].apply(len)
df_sinonimos.sort_values(by='N_sinonimos',ascending=False,inplace=True)
df_sinonimos.to_excel('sinonimos_palabras_clave.xlsx',index=False)

## Método 2: WordReference scrapping

In [ ]:
def sinonimos_wordreference(word):
    url='http://www.wordreference.com/sinonimos/'
    buscar=url+word
    resp=requests.get(buscar)
    bs=BeautifulSoup(resp.text,'lxml')
    lista=bs.find_all(class_='trans clickable')
    for sin in lista:
        sino=sin.find_all('li')
    return sino[0].next_element.split(',')

# Sentence clustering

In [ ]:
activities_list = df_planificacion_total['Nombre Actividad'].unique()
activities_list = [str(x) for x in activities_list]
activities_list_test = activities_list[:1000]

#look_for_clusters(eng_sentences, similarity_threshold=60)
cluster_list = look_for_clusters(activities_list_test, language="spanish", method="token_set_ratio", similarity_threshold=90)

# dataframe 
df_clusters = pd.DataFrame(list(zip(activities_list_test,cluster_list)),
               columns =['Nombre_actividad', 'cluster'])

for i in range(1,100):
    print(df_clusters[df_clusters.cluster==i].Nombre_actividad.to_list(),'\n')

# Topic modeling

https://medium.com/@jarch/topic-modeling-with-spanish-text-an-introductory-example-in-python-9cdb10fbe126

In [ ]:
df_planificacion_total["texto_total"] = df_planificacion_total['nombre_actividad'].astype(str) + ' ' + df_planificacion_total['descripción_del_plan'].astype(str)
# Spanish text analysis tools 
stopwords = set(nltk.corpus.stopwords.words("spanish"))
nlp = spacy.load("es_core_news_sm")
def normalize_tokenize_lemmatize(text):
    doc = nlp(text)
    VALID_POS = {'PROPN', 'NOUN'}
    # Avoid empty word in beggining with doc[1:]
    return [
        word.lemma_ for word in doc if word.lemma_ not in stopwords
        and word.pos_ in VALID_POS
        ]
# This step may take a few minutes
df_planificacion_total['texto_total_norm'] = df_planificacion_total['texto_total'].apply(normalize_tokenize_lemmatize)

In [ ]:
popular_activities = list(df_planificacion_total['nombre_actividad'].dropna().value_counts().index)[:50]
lista_descripcion = list(df_planificacion_total[df_planificacion_total.nombre_actividad.isin(popular_activities)].descripción_del_plan.values)
text_for_dict = [normalize_tokenize_lemmatize(str(x)) for x in lista_descripcion]

In [ ]:
dictionary = Dictionary(text_for_dict)
# Assign ids to every word without gaps
dictionary.compactify()
# Filter out words that appear in less than 2 documents, thos appearing in more than 97% of corpus, and keep all other words
dictionary.filter_extremes(no_below=2, no_above=0.97, keep_n=None)
# Reassign ids to every word without gaps after filtering
dictionary.compactify()
# For every review, create a B.O.W. representation
corpus = [dictionary.doc2bow(text) for text in text_for_dict]

In [ ]:
# Find the optimal number of topics based on coherence
df_planificacion_total['nombre_actividad'] = df_planificacion_total['nombre_actividad'].astype(str)
texts = list(df_planificacion_total['nombre_actividad'].unique())[:1000]

MAX_TOPICS = 20
coherence_vals = []
lm_list = []
# This loop may take a while
for n_topics in range(10, MAX_TOPICS+1):
    print(n_topics)
    lm = LdaModel(corpus=corpus, num_topics = n_topics,
          id2word=dictionary)
    lm_list.append(lm)
    
    cm = CoherenceModel(model=lm, texts=texts,
         dictionary=dictionary, coherence='c_v')
    
    coherence_vals.append(cm.get_coherence())

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))
ax.plot(range(10, MAX_TOPICS+1), coherence_vals)
ax.set_xlabel("N° de tópicos")
ax.set_ylabel("Coherencia")
ax.set_xticks(range(1,21))
plt.grid(True)
plt.show()

# Similarity

In [ ]:
# download language model: python -m spacy download es_core_news_sm 
nlp = spacy.load("es_core_news_md")
#stopwords_esp = nltk.corpus.stopwords.words('spanish')               # Cargamos las palabras huecas en una lista

In [ ]:
doc1 = nlp("Trayectoria")
texts = data['Nombre Actividad'].unique()
texts = texts[:1000]

# remove stopwords
texts = [x.split() for x in texts]
texts = [[y for y in x if y not in stopwords_esp] for x in texts]
texts = [' '.join(x) for x in texts]

i = 0
list_sim = []
for text in texts:
    doc2 = nlp(text)
    sim = doc1.similarity(doc2)
    list_sim +=[[i,sim]]
    i+=1

# PhraseMatcher 

In [ ]:
# method for searching keyword from the text
def search_for_keyword(keyword, doc_obj, nlp):
    phrase_matcher = PhraseMatcher(nlp.vocab)
    phrase_list = [nlp(keyword)]
    phrase_matcher.add("Text Extractor", None, *phrase_list)

    matched_items = phrase_matcher(doc_obj)

    matched_text = []
    for match_id, start, end in matched_items:
        text = nlp.vocab.strings[match_id]
        span = doc_obj[start: end]
        matched_text.append(span.sent.text)
    return matched_text

In [ ]:
X = search_for_keyword('trayectorias', nlp('. '.join(texts)), nlp)

# BERT

In [ ]:
# load our Sentence Transformers model pre trained!!
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [ ]:
# this is not production ready data!!
sentences = [sentence.lower().replace('br','').replace('<',"").replace(">","").replace('\\',"").replace('\/',"") 
             for sentence in data['Descripción del Plan'].sample(n=2000)]

#see a sentence, and our length
print(sentences[5:6], f'\n\nLength Of Data {len(sentences)}')

In [ ]:
# lets find the semantically closest sentence to a random sentence
# that we come up with, in our dataset

# i like action movies, mission impossible is one of my favorites
our_sentence = 'Trayectoria'

# lets embed our sentence
my_embedding = model.encode(our_sentence)

# lets embed the corpus
#embeddings = model.encode(sentences)

#Compute cosine similarity between my sentence, and each one in the corpus
cos_sim = util.cos_sim(my_embedding, embeddings)

# lets go through our array and find our best one!
# remember, we want the highest value here (highest cosine similiarity)
winners = []
for arr in cos_sim:
    for i, each_val in enumerate(arr):
        winners.append([sentences[i],each_val])

# lets get the top 2 sentences
final_winners = sorted(winners, key=lambda x: x[1], reverse=True)



for arr in final_winners[0:2]:
    print(f'\nScore : \n\n  {arr[1]}')
    print(f'\nSentence : \n\n {arr[0]}')

# SentenceTransformers 

In [ ]:
# Cargar Sentence Transformers model preentrenado y apto para español
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [ ]:
# cargar bbdd
df_planificacion_total = pd.read_csv('planificacion_anual_parte1_2018_2022.csv', low_memory=False)
# Frases a embeder
sentences_descripcion_plan = df_planificacion_total['Descripción del Plan'].values
# lets embed the corpus: 16 HORAS 
# embeddings = model.encode(sentences_descripcion_plan)
# guardar 
# np.save(r'C:\Users\JoaquinFarina\Desktop\Tether\embeddings_sentences_descripcion_plan.npy', embeddings)
embeddings = np.load(r'C:\Users\JoaquinFarina\Desktop\Tether\embeddings_sentences_descripcion_plan.npy')

In [ ]:
# cargar categorizaciones pre establecidas 
data_cat = pd.read_excel(r'C:\Users\JoaquinFarina\Desktop\Tether\Listado palabras y conceptos Esudio PME FORMATED.xlsx')

In [ ]:
sentences = sentences_descripcion_plan

In [ ]:
# Encontrar descripciones más cercanas a cada concepto
lista_terminos = list(data_cat['Palabras/Conceptos'])

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('clasificacion_embedding_descripcion_actividad.xlsx', engine='xlsxwriter')

for sentence in lista_terminos: 
    print('\n',sentence)
    sentence = sentence.replace('“', '')
    sentence = sentence.replace('”', '')
    sentence = sentence.replace(';', ',')
    sentence = sentence.replace('*', ',')
    sentence = sentence.replace(':', ',')
    
    # embeder concepto
    my_embedding = model.encode(sentence)
    # calcular similaridad de coseno entre concepto y corpus
    cos_sim = util.cos_sim(my_embedding, embeddings)

    # lets go through our array and find our best one!
    # remember, we want the highest value here (highest cosine similiarity)
    winners = []
    for arr in cos_sim:
        for i, each_val in enumerate(arr):
            winners.append([sentences[i],each_val])

    # lets get the top 2 sentences
    final_winners = sorted(winners, key=lambda x: x[1], reverse=True)
    print(final_winners[:2],'\n')
    
    # Write each dataframe to a different worksheet.
    pd.DataFrame(final_winners[:50000], columns = ['Descripción Actividad','score']) .to_excel(writer, sheet_name=sentence[:30])

# Close the Pandas Excel writer and output the Excel file.
writer.close()

## Results

In [ ]:
# for pandas version >= 0.21.0
sheet_to_df_map = pd.read_excel('clasificacion_embedding_descripcion_actividad_annotated.xlsx', sheet_name=None)

In [ ]:
dict_nombres_cat = {"Lenguaje y comunicación, Lengu": "Lenguaje y comunicación",
                    "Inglés, idioma": "Inglés",
                    "Matemática": "Matemática",
                    "Ciencias Naturales, Biología, " : "Ciencias",
                    "Historia, Geografía, Ciencias ": "Historia",
                    "Educación Física, salud, depor": "Educación Física",
                    ", Arte,, Música, musical, Visu": "Arte/Música",
                    "Lengua y Cultura de los Pueblo": "Pueblos originarios",
                    "ciudadanía digital, uso de tec": "Uso tecnología",
                    "pandemia, crisis, sanitaria, C": "Pandemia",
                    "convivencia, convivencia esco" : "Convivencia",
                    "género, diversidad, diversidad": "Género/diversidad"} 

In [ ]:
j = 0
for cat in sheet_to_df_map: 
    df = sheet_to_df_map[cat]
    if ('umbral' in df.columns) and ('migra' not in cat) :
        cat2 = cat.replace('"', '')
        print(dict_nombres_cat[cat2])
        
        # procesar df 
        # identificar última fila a consideerar 
        i_umbral = df[df.umbral == 1].index[0]
        # filtrar df
        df_final = df[df.index<=i_umbral]
        # eliminar nulos
        df_final.dropna(subset = ['Descripción Actividad'],inplace=True)
        # eliminar duplicados
        df_final.drop_duplicates(subset = ['Descripción Actividad'],inplace=True)
        # definir columna de conteo 
        df_final.loc[:,'CAT ' + dict_nombres_cat[cat2]] = 1
        # acotar columnas
        df_final = df_final[['Descripción Actividad','CAT ' + dict_nombres_cat[cat2]]]
        # homologar nombre columna
        df_final.rename(columns = {'Descripción Actividad':'Descripción del Plan'}, inplace=True)
        
        if j == 0:
            df_planificacion_total_CAT = pd.merge(df_planificacion_total,df_final,  on = 'Descripción del Plan', how = 'left')
        else:
            df_planificacion_total_CAT = pd.merge(df_planificacion_total_CAT,df_final,  on = 'Descripción del Plan', how = 'left')
        j+=1
        
        print(len(df_final))
            

In [ ]:
def columnas_agregacion(x):
    d = {}
    d['Actividades por EE'] = np.round(x['rbd'].count()/x['rbd'].nunique(),1)
    return pd.Series(d, index=['Actividades por EE'])

In [ ]:
def calcular_actividades_por_EE_variable(variable):
    for a in range(2018,2023):
        if a == 2018:
            df_tot = df_planificacion_total_CAT[df_planificacion_total_CAT.agno == a][[variable,'rbd']].groupby([variable]).apply(columnas_agregacion)
            df_tot.columns = [str(a) for x in df_tot.columns]
        else:
            s2 = df_planificacion_total_CAT[df_planificacion_total_CAT.agno == a][[variable,'rbd']].groupby([variable]).apply(columnas_agregacion)
            s2.columns = [str(a) for x in s2.columns]
            df_tot = pd.concat([df_tot,s2], axis = 1)
    return df_tot

In [ ]:
list_total_EE = []
list_total_prom = [] 
for col in df_planificacion_total_CAT.columns: 
    if 'CAT' in col:
        print(col)
        list_cat_EE = []
        list_cat_prom = []
        for a in range(2018,2023):
            df_agno = df_planificacion_total_CAT[df_planificacion_total_CAT.agno == a]
            # Actividades por EE
            N_EE = len(df_agno[df_agno[col] == 1])/len(df_agno.rbd.unique()) 
            # promedio de actividades 
            N_prom = len(df_agno[df_agno[col] == 1])/len(df_agno) 
            
            list_cat_EE += [np.round(N_EE,2)]
            list_cat_prom += [100*np.round(N_prom,3)]
            
        list_total_EE += [[col[4:]] + list_cat_EE]
        list_total_prom += [[col[4:]] + list_cat_prom]

In [ ]:
df_total_EE = pd.DataFrame(list_total_EE, columns = ['Categoría', 2018,2019,2020,2021,2022])
df_total_EE

In [ ]:
df_total_prom = pd.DataFrame(list_total_prom, columns = ['Categoría', 2018,2019,2020,2021,2022])
df_total_prom

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(r'C:\Users\JoaquinFarina\Dropbox\Team NLP\Data\Planes de Mejoramiento Educativo\Informe1\Tablas_informe1\tablas_categorzaciones.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_total_EE.to_excel(writer, sheet_name='actividades por EE')
df_total_prom.to_excel(writer, sheet_name='% de actividades')

# Close the Pandas Excel writer and output the Excel file.
writer.close()

In [ ]:
df_planificacion_total_CAT['CAT_suma'] = 0
for col in df_planificacion_total_CAT.columns: 
    if ('CAT' in col) and ('suma' not in col):
        print(col)
        df_planificacion_total_CAT['CAT_suma'] = df_planificacion_total_CAT['CAT_suma'] + df_planificacion_total_CAT[col].fillna(0)

In [ ]:
for a in range(2018,2023):
    df_a = df_planificacion_total_CAT[df_planificacion_total_CAT.agno==a]
    print(a)
    if a == 2018:
        df_resumen_suma_act = np.round(100*df_a.CAT_suma.value_counts().sort_index()/len(df_a),2)
    else:
        df_resumen_suma_act = pd.concat([df_resumen_suma_act,np.round(100*df_a.CAT_suma.value_counts().sort_index()/len(df_a),2) ],axis = 1)

In [ ]:
df_resumen_suma_act.columns = [2018,2019,2020,2021,2022]
df_resumen_suma_act.to_excel(r'Informe1\Tablas_informe1\df_resumen_suma_actividades.xlsx')
df_resumen_suma_act

In [ ]:
df_planificacion_total_CAT.to_csv(r'planificacion_anual_parte1_2018_2022_EDA_classification_FINAL.csv')